In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Load the CSV containing questions, answers, and categories (adjust the file path as necessary)
faq_df = pd.read_csv('resources/answers.csv')  # Replace with the correct file path

# Combine both questions and answers into a single column for analysis
faq_df['Combined'] = faq_df['Questions'] + " " + faq_df['Answers']

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Vectorize the combined questions and answers
X = vectorizer.fit_transform(faq_df['Combined'])

# Use 'Category' as labels (Y)
Y = faq_df['Category']

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train, Y_train)

# Evaluate the model's accuracy
Y_pred = model.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

def suggest_questions_based_on_model(user_input):
    # Transform the user's input into the TF-IDF vector space
    user_input_vector = vectorizer.transform([user_input])
    
    # Predict the category using the trained model
    predicted_category = model.predict(user_input_vector)[0]
    print(f"\nPredicted Category: {predicted_category}")
    
    # Filter FAQ by the predicted category
    category_faqs = faq_df[faq_df['Category'] == predicted_category]
    
    # Compute the cosine similarity between the user's input and the predefined questions + answers
    category_faq_vectors = vectorizer.transform(category_faqs['Combined'])
    similarities = cosine_similarity(user_input_vector, category_faq_vectors)
    
    # Sort the questions based on the similarity scores in descending order
    similar_indices = similarities.argsort()[0][::-1]
    
    # Get the top 3 most similar questions
    top_3_indices = similar_indices[:3]
    
    suggested_questions = category_faqs['Questions'].iloc[top_3_indices]
    
    return suggested_questions, category_faqs.iloc[top_3_indices]

# Keep looping to ask questions until the user decides to quit
while True:
    # Get user input
    user_question = input("\nAsk me a question (or type 'exit' to quit): ")

    # Break the loop if the user types 'exit' or 'quit'
    if user_question.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break

    # Get suggested questions based on user's input and model prediction
    suggested_questions, selected_faqs = suggest_questions_based_on_model(user_question)

    # Display suggested questions to the user
    print("\nHere are some questions I can answer. Please select one by number:")
    for idx, question in enumerate(suggested_questions, start=1):
        print(f"{idx}. {question}")

    # Simulate the user selecting a question
    try:
        selected_option = int(input("\nPlease select the number of the question you want answered: ")) - 1
        
        # Ensure valid input
        if selected_option not in range(len(suggested_questions)):
            print("Invalid selection. Please choose a valid number.")
            continue

        # Retrieve the corresponding answer from the filtered category DataFrame
        answer = selected_faqs['Answers'].iloc[selected_option]

        print(f"\nAnswer: {answer}")
    except ValueError:
        print("Invalid input. Please enter a number corresponding to a question.")

Model Accuracy: 65.12%

Predicted Category: activities

Here are some questions I can answer. Please select one by number:
1. How does the Fat Tire Electric Bike Tour create a memorable experience for visitors?
2. How can visitors interact with and feed Southern Rays during the Stingray City Adventure?
3. What are some of the key experiences offered by highly personalized luxury tours in Antigua?
Invalid input. Please enter a number corresponding to a question.

Predicted Category: activities

Here are some questions I can answer. Please select one by number:
1. How can visitors interact with and feed Southern Rays during the Stingray City Adventure?
2. What are some of the key experiences offered by highly personalized luxury tours in Antigua?
3. How does GeoTourist enhance visitor experiences with guided audio tours?

Answer: Fat Tire Electric Bike Tour creates an intimate guided tour experience, utilizing comfortable, easy-to-ride electric beach cruisers. The best way to tour, explo